## This juypter notebook is tartget towards defining alleles 

The idea is that alleles are on corresponding haplotig and a good blast hit.

map all h contigs on their corresponding p alleles. Use the Assembletic '*oriented_coords.csv' file for this purpose.
This needs to be converted to a bed file or a gff file in order to pull out the overlap between gene models and the h_contig using either bedtools or pandas. Check what overlaps can spit out on the other side. 
>Considerations here are what happens if nothing overlaps or what if muliptle overlaps. 

Do gene blast of p on h with certain cut off. Pull out the best hit that is located on the corresponding haplotig. If that is not there look for best hit on other haplotigs or primary contigs that is not itself.

>Considerations are that p-h parings that are best hits and corresponding haplotig overule all others. There can only be a single hit. 



In [1]:
%matplotlib inline

In [6]:
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil


In [47]:
#Define the PATH
BASE_AA_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12'
BASE_A_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
ASSEMBLETICS_INPATH = os.path.join(BASE_AA_PATH, 'Assembletics')
BLAST_DB = os.path.join(BASE_AA_PATH, 'blast_DB')
OUT_PATH = os.path.join(BASE_AA_PATH, 'allele_analysis')
OUT_tmp = os.path.join(OUT_PATH, 'tmp')
if not os.path.isdir(OUT_PATH):
    os.mkdir(OUT_PATH)
if not os.path.isdir(OUT_tmp):
    os.mkdir(OUT_tmp)

In [12]:
#Define your p and h genome and move it into the allele analysis folder
p_genome = 'Pst_104E_v12_p_ctg'
h_genome = 'Pst_104E_v12_h_ctg'
genome_file_suffix = '.genome_file'
for x in (x + '.fa' for x in [p_genome, h_genome]):
    shutil.copy2(BASE_A_PATH+'/'+x, OUT_tmp)

In [138]:
def att_column_p(x, y, z, a, b):
    '''Generate attribute column of h on p p_gff dataframe.
    '''
    string = 'h_contig=%s;query_start=%s;query_stop=%s;query_length=%s;query_aln_ln=%s' % (x, str(y), str(z), str(a), str(b))
    return string

In [139]:
def att_column_h(x, y, z, a, b):
    '''Generate attribute column of h on p h_gffdataframe.
    '''
    string = 'p_contig=%s;ref_start=%s;ref_stop=%s;ref_length=%s;ref_aln_ln=%s' % (x, str(y), str(z), str(a), str(b))
    return string

In [140]:
def gene_contig_subset(df, contig, feature):
    '''Input the gff and subset the feature columne by gene.
        Return data frame subset by contig and gene'''
    tmp_df = df[(df[0] == contig) & (df[2] == feature)]
    return tmp_df

In [141]:
#get assembletics folders
ass_folders = [os.path.join(ASSEMBLETICS_INPATH, x) for x in os.listdir(ASSEMBLETICS_INPATH) if x.endswith('_php_8kbp')]
ass_folders.sort()

In [144]:
for folder in ass_folders[:1]:
    orient_coords_file = [os.path.join(folder, x) for x in os.listdir(folder) if x.endswith('oriented_coords.csv')][0]
    #load in df and generate several additions columns
    tmp_df = pd.read_csv(orient_coords_file, sep=',')
    tmp_df['p_feature'] = "haplotig"
    tmp_df['h_feature'] ='primary_contig'
    tmp_df['strand'] = "+"
    tmp_df['frame'] = 0
    tmp_df['query_aln_len'] = abs(tmp_df['query_start']-tmp_df['query_length'])
    tmp_df['alignment_length'] = abs(tmp_df['ref_end'] - tmp_df['ref_start'])
    tmp_df.reset_index(drop=True, inplace=True)
    tmp_df.sort_values('query', inplace=True)
    tmp_df['attribute_p'] = tmp_df.apply(lambda row: att_column_p(row['query'],row['query_start'], row['query_end'],\
                                                         row['query_length'], row['query_aln_len']), axis=1)
    
    tmp_df['attribute_h'] = tmp_df.apply(lambda row: att_column_h(row['ref'], row['ref_start'], row['ref_end'],\
                                                                  row['ref_length'], row['alignment_length']), axis=1)
    
    #generate tmp gff dataframe
    tmp_p_gff_df = tmp_df.loc[:, ['ref', 'query', 'p_feature','ref_start','ref_end', 'alignment_length', 'strand', 'frame', 'attribute_p']]
    tmp_p_gff_df.sort_values('ref_start',inplace=True)
    tmp_h_gff_df = tmp_df.loc[:, ['query', 'ref', 'h_feature', 'query_start', 'query_end', 'query_aln_len', 'strand', 'frame', 'attribute_h']]
    tmp_h_gff_df.sort_values('query_start', inplace=True)
    
    #write those out to new together with the p and h
    

In [145]:
tmp_p_gff_df

,ref,query,p_feature,ref_start,ref_end,alignment_length,strand,frame,attribute_p
10,pcontig_000,hcontig_000_003,haplotig,4096,6881,2785,+,0,h_contig=hcontig_000_003;query_start=1;query_s...
11,pcontig_000,hcontig_000_003,haplotig,10766,14593,3827,+,0,h_contig=hcontig_000_003;query_start=6520;quer...
12,pcontig_000,hcontig_000_003,haplotig,14589,65671,51082,+,0,h_contig=hcontig_000_003;query_start=19775;que...
141,pcontig_000,hcontig_000_031,haplotig,21297,24866,3569,+,0,h_contig=hcontig_000_031;query_start=1;query_s...
4,pcontig_000,hcontig_000_031,haplotig,24879,45212,20333,+,0,h_contig=hcontig_000_031;query_start=3551;quer...
13,pcontig_000,hcontig_000_003,haplotig,71086,109668,38582,+,0,h_contig=hcontig_000_003;query_start=120909;qu...
14,pcontig_000,hcontig_000_003,haplotig,109421,121280,11859,+,0,h_contig=hcontig_000_003;query_start=159489;qu...
142,pcontig_000,hcontig_000_003,haplotig,124057,132521,8464,+,0,h_contig=hcontig_000_003;query_start=169758;qu...
143,pcontig_000,hcontig_000_003,haplotig,131654,132560,906,+,0,h_contig=hcontig_000_003;query_start=182219;qu...
15,pcontig_000,hcontig_000_003,haplotig,132560,173947,41387,+,0,h_contig=hcontig_000_003;query_start=177431;qu...


In [127]:
tmp_df.reset_index(drop=True, inplace=True)

In [128]:
tmp_df['feature'] = "haplotig"
tmp_df['strand'] = "+"
tmp_df['frame'] = 0
tmp_df['query_aln_len'] = abs(tmp_df['query_start']-tmp_df['query_length'])

In [129]:
tmp_df['attribute'] = tmp_df.apply(lambda row: att_column(row['query'],row['query_start'], row['query_end'],\
                                                         row['query_length'], row['query_aln_len']), axis=1)

In [130]:
tmp_df.head()

,ref_start,ref_end,query_start,query_end,ref_length,query_length,ref,query,tag,alignment_length,feature,strand,frame,query_aln_len,attribute
0,1723814,1738366,1718554,1733091,3073024,2066231,pcontig_000,hcontig_000_003,unique,14552,haplotig,+,0,347677,h_contig=hcontig_000_003;query_start=1718554;q...
1,1949361,1950329,192710,193679,3073024,2066231,pcontig_000,hcontig_000_003,repetitive,968,haplotig,+,0,1873521,h_contig=hcontig_000_003;query_start=192710;qu...
2,1956358,1958369,1257435,1255409,3073024,2066231,pcontig_000,hcontig_000_003,repetitive,2011,haplotig,+,0,808796,h_contig=hcontig_000_003;query_start=1257435;q...
3,1958941,1959909,192710,193679,3073024,2066231,pcontig_000,hcontig_000_003,repetitive,968,haplotig,+,0,1873521,h_contig=hcontig_000_003;query_start=192710;qu...
4,1958853,1959909,1254958,1253909,3073024,2066231,pcontig_000,hcontig_000_003,repetitive,1056,haplotig,+,0,811273,h_contig=hcontig_000_003;query_start=1254958;q...


In [131]:
tmp_gff_df = tmp_df.loc[:, ['ref','query', 'feature','ref_start','ref_end', 'alignment_length', 'strand', 'frame', 'attribute']]

In [132]:
tmp_gff_df.iloc[0,8]

'h_contig=hcontig_000_003;query_start=1718554;query_stop=1733091;query_length=2066231;query_aln_ln=347677'

In [35]:
#generate the blast databases if not already present
os.chdir(BLAST_DB)
blast_dir_content = os.listdir(BLAST_DB)
for x in blast_dir_content:
    if x.endswith('.fa') and ({os.path.isfile(x + e) for e in ['.psq', '.phr', '.pin'] } != {True}\
           and {os.path.isfile(x + e) for e in ['.nin', '.nhr', '.nsq'] } != {True} ):

        make_DB_options = ['-in']
        make_DB_options.append(x)
        make_DB_options.append('-dbtype')
        if 'protein' in x:
            make_DB_options.append('prot')
        else:
            make_DB_options.append('nucl')
        make_DB_command = 'makeblastdb %s' % ' '.join(make_DB_options)
        make_DB_stderr = subprocess.check_output(make_DB_command, shell=True, stderr=subprocess.STDOUT)
        print('%s is done!' % make_DB_command)
print("All databases generated and ready to go!")

All databases generated and ready to go!
